In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import gzip
import json
import pandas as pd
import fileinput
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, ArrayType
# import pyspark config
from pyspark import SparkConf



In [5]:

# create spark session builder for elastic search using username and password as basic auth and certifications
spark = SparkSession.builder.appName("ecommerce_recommendation_system").\
    config("es.nodes", "localhost").\
    config("es.port", "9200").\
    config("es.index.auto.create", "true").\
    config("spark.es.net.http.auth.user", "elastic").\
    config("spark.jars", "jars/elasticsearch/elasticsearch-hadoop-8.8.0/dist/elasticsearch-spark-20_2.11-8.8.0.jar"). \
    config("spark.es.net.http.auth.pass", "trwI3C-KciiNdfz24a5q").\
    getOrCreate()
# test spark_es has successfully connected to elastic search
# spark_es.sql("SELECT 1").show()
spark.sql("SELECT 1").show()


+---+
|  1|
+---+
|  1|
+---+



In [6]:
# Specify the path to your gzip-compressed metadata file
metadata_file = "/Users/gracechongzuting/Documents/revision_daily/PySparkML/meta_Appliances.json.gz"

# Replace single quotes with double quotes
with gzip.open(metadata_file, 'rt') as file:
    for line in file:
        pass
        # print(line.replace("'", '"'), end='')

# Read in the metadata file as a DataFrame 
df = spark.read.json(metadata_file)
# create pandas dataframe
df = df.toPandas()
df.head(5)

,asin,brand,categories,description,imUrl,price,related,salesRank,title
0,0970408285,None,"[[Appliances, Parts & Accessories, Washer Part...",High-pressure hose 90 end & hose washer instal...,http://ecx.images-amazon.com/images/I/41kQjPIF...,40.75,None,None,None
1,7301113188,None,"[[Appliances, Freezers, Chest Freezers]]",None,http://ecx.images-amazon.com/images/I/41EfFOPw...,NaN,None,None,None
2,B00002N7HY,Leviton,"[[Appliances, Parts & Accessories, Range Parts...","Built of durable thermoplastic, Leviton Power ...",http://ecx.images-amazon.com/images/I/316Y1KbT...,2.29,"([B000BPFZHM, B000VU1KC6, B00004YUMZ, B000FK9X...",None,"Leviton 5050 50 Amp, 125/250 Volt, NEMA 10-50R..."
3,B00002NATH,Leviton,"[[Appliances, Parts & Accessories, Range Parts...","Built of durable thermoplastic, Leviton Power ...",http://ecx.images-amazon.com/images/I/41srC2QF...,9.13,"([B002OUMU66, B007HFT034, B000EDQR6E, B00192MU...",None,"Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P..."
4,B00004SQH9,Coleman Cable,"[[Appliances, Parts & Accessories, Refrigerato...",Coleman Cable 3536 14/3 General-Use Appliance ...,http://ecx.images-amazon.com/images/I/41eg0Hvy...,16.29,"([B000XSNDS0, B0022NH420, B000BQT7B6, B00004SQ...",None,Coleman Cable 3536 14/3 General-Use Appliance ...


In [7]:
# only show the columns we need 
df = df[['asin', 'title', 'brand', 'imUrl', 'categories', 'price']]
# change column name or imUrl to image
df = df.rename(columns={'imUrl': 'image'})
# only extract the last index of categories column and rename to main category
df['main_category'] = df['categories'].apply(lambda x: x[-1][-1])
# drop duplicates from asin column
df = df.drop_duplicates(subset=['asin'])


df.head(5)

,asin,title,brand,image,categories,price,main_category
0,0970408285,None,None,http://ecx.images-amazon.com/images/I/41kQjPIF...,"[[Appliances, Parts & Accessories, Washer Part...",40.75,Washer Parts & Accessories
1,7301113188,None,None,http://ecx.images-amazon.com/images/I/41EfFOPw...,"[[Appliances, Freezers, Chest Freezers]]",NaN,Chest Freezers
2,B00002N7HY,"Leviton 5050 50 Amp, 125/250 Volt, NEMA 10-50R...",Leviton,http://ecx.images-amazon.com/images/I/316Y1KbT...,"[[Appliances, Parts & Accessories, Range Parts...",2.29,Plug Receptacles
3,B00002NATH,"Leviton 7313 30 Amp, 125 Volt, NEMA Tt-30R, 2P...",Leviton,http://ecx.images-amazon.com/images/I/41srC2QF...,"[[Appliances, Parts & Accessories, Range Parts...",9.13,Plug Receptacles
4,B00004SQH9,Coleman Cable 3536 14/3 General-Use Appliance ...,Coleman Cable,http://ecx.images-amazon.com/images/I/41eg0Hvy...,"[[Appliances, Parts & Accessories, Refrigerato...",16.29,Power Cords


In [8]:
sdf = spark.createDataFrame(df)


In [9]:
sdf.groupBy("main_category").count().orderBy(col('count').desc()).show(100)

+--------------------+-----+
|       main_category|count|
+--------------------+-----+
| Parts & Accessories| 1236|
|       Water Filters| 1046|
|       Refrigerators|  795|
|Range Parts & Acc...|  541|
|Built-In Dishwashers|  510|
|             Washers|  505|
|         Range Hoods|  488|
|Refrigerator Part...|  450|
| Freestanding Ranges|  385|
|            Cooktops|  343|
|              Dryers|  319|
|Dryer Parts & Acc...|  272|
|         Power Cords|  219|
|          Ice Makers|  214|
|              Motors|  210|
| Replacement Filters|  182|
|Cooktop Parts & A...|  180|
|          Appliances|  164|
|           Wash Arms|  162|
|     Humidity Meters|  161|
|Oven Parts & Acce...|  159|
|           Trim Kits|  148|
|           Fasteners|  143|
|               Knobs|  143|
|               Vents|  138|
|Dishwasher Parts ...|  137|
|    Heating Elements|  130|
|   Replacement Wicks|  125|
|             Filters|  121|
|               Doors|  110|
|Washer Parts & Ac...|  100|
|             

In [10]:
from elasticsearch import Elasticsearch

In [11]:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html
es = Elasticsearch('https://localhost:9200', ca_certs='./http_ca.crt', basic_auth=('elastic', 'trwI3C-KciiNdfz24a5q'))
# reset certificate to connect to docker elastic
# reset certificate because by default it will expire after 7 days
# certifcate expired after 7 days
# reset password to elastic
es.info(pretty=True)
# doc = {
#     'author': 'author_name',
#     'text': 'Interensting content...',
#     'timestamp': datetime.now(),
# }
# resp = es.index(index="test-index", id=1, document=doc)
# print(resp['result'])


ObjectApiResponse({'name': 'c4d1055f0816', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tmqFsIZfR-OF3wHvX3tikw', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
# delete purpose is to reset the index for each time we run the code
es.indices.delete(index="products")
VECTOR_DIM = 25
#  25-dimensional vector representing a person's characteristics, 
# each dimension could represent attributes like age, height, weight, income



product_map = {
    "mappings": {
        "properties": {
            "asin": {
                "type": "keyword"
            },
            "title": {
                "type": "text"
            },
            "image": {
                "type": "keyword"
            },
            "brand": {
                "type": "keyword"
            },
            "categories": {
                "type": "keyword"
            },
            "main_category": {
                "type": "keyword"
            },
            "price" : {
                "type": "float",
                "null_value": 0.0
            },
            # to define model vectors and metadata
            "model_vector": {
                "type": "dense_vector",
                "dims": VECTOR_DIM
            },
            "model_version": {
                "type": "keyword"
            },
            "model_timestamp": {
                "type": "date"
            }
        }
    }
}

resultant = es.indices.create(index="products", body=product_map)
print(resultant)

/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/755845177.py:49: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resultant = es.indices.create(index="products", body=product_map)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [13]:
sdf.show(5)

+----------+--------------------+-------------+--------------------+--------------------+-----+--------------------+
|      asin|               title|        brand|               image|          categories|price|       main_category|
+----------+--------------------+-------------+--------------------+--------------------+-----+--------------------+
|0970408285|                null|         null|http://ecx.images...|[[Appliances, Par...|40.75|Washer Parts & Ac...|
|7301113188|                null|         null|http://ecx.images...|[[Appliances, Fre...|  NaN|      Chest Freezers|
|B00002N7HY|Leviton 5050 50 A...|      Leviton|http://ecx.images...|[[Appliances, Par...| 2.29|    Plug Receptacles|
|B00002NATH|Leviton 7313 30 A...|      Leviton|http://ecx.images...|[[Appliances, Par...| 9.13|    Plug Receptacles|
|B00004SQH9|Coleman Cable 353...|Coleman Cable|http://ecx.images...|[[Appliances, Par...|16.29|         Power Cords|
+----------+--------------------+-------------+-----------------

In [14]:
# SHOW all distinct values for price column
sdf.select('price').distinct().show()

+------+
| price|
+------+
|  9.13|
|160.36|
| 12.32|
|  15.5|
| 46.15|
| 19.98|
|  8.51|
|  5.86|
| 16.54|
| 21.89|
| 21.52|
| 14.75|
|  5.48|
|   8.0|
|  9.49|
|  18.3|
|  7.91|
| 18.03|
| 840.0|
| 12.79|
+------+
only showing top 20 rows



In [15]:

from datetime import datetime
# # Define the index name
index_name = "products"

VECTOR_DIM = 25

# Check if DataFrame is empty
if df.empty:
    print("DataFrame is empty. No data to index.")
    exit()

# Iterate over the DataFrame rows and index each document
for index, row in df.iterrows():
    doc = {
        'asin': row['asin'],
        'title': row['title'],
        'brand': row['brand'],
        'image': row['image'],
        'categories': row['categories'],
        'main_category': row['main_category'],
        'price': row['price'],
        'model_timestamp': datetime.now()
    }
    # print(doc)
    # if price is NaN, replace with 0.0
    if pd.isnull(doc['price']):
        doc['price'] = 0.0
    resp = es.index(index=index_name, id=row['asin'], body=doc)

# Refresh the index to make the documents searchable immediately
#es.indices.refresh(index=index_name)

# Check if data is inserted into products 
search_response = es.search(index=index_name, body={"query": {"match_all": {}}})
hits = search_response["hits"]["hits"]
if len(hits) > 0:
    print("Data inserted successfully.")
else:
    print("No data found in the index.")


/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/2449024641.py:28: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(index=index_name, id=row['asin'], body=doc)


/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/2449024641.py:34: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_response = es.search(index=index_name, body={"query": {"match_all": {}}})


Data inserted successfully.


In [16]:
# search for a product with title like Coleman Cable 353
search_response = es.search(index=index_name, body={"query": {"match": {"title": "Coleman Cable 353"}}})
search_response.body

/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/1041343867.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_response = es.search(index=index_name, body={"query": {"match": {"title": "Coleman Cable 353"}}})


{'took': 81,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 19, 'relation': 'eq'},
  'max_score': 10.828733,
  'hits': [{'_index': 'products',
    '_id': 'B002G9TNII',
    '_score': 10.828733,
    '_source': {'asin': 'B002G9TNII',
     'title': 'Coleman Cable 09733 16/3 SPT-3 Replacement Power Supply Cable, 13-Amp 125-Volt 3-Feet, Gray',
     'brand': 'Coleman Cable',
     'image': 'http://ecx.images-amazon.com/images/I/31pgU8c5sgL._SY300_.jpg',
     'categories': [['Appliances',
       'Parts & Accessories',
       'Freezer Parts & Accessories',
       'Power Cords']],
     'main_category': 'Power Cords',
     'price': 2.1,
     'model_timestamp': '2023-06-14T12:26:30.103132'}},
   {'_index': 'products',
    '_id': 'B00004SQH9',
    '_score': 10.783002,
    '_source': {'asin': 'B00004SQH9',
     'title': 'Coleman Cable 3536 14/3 General-Use Appliance Extension Cord, 15-Feet',
     'brand': 'Coleman Cable',
     '

In [17]:
es.count(index="products").body

{'count': 11656,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [18]:
# check products count and elastic search count to ensure they are the same
print("products spark dataframe count: ", sdf.count())
print("elastic search products index count: ", es.count(index="products").body['count'])

products spark dataframe count:  11656
elastic search products index count:  11656


In [19]:
# lets search for a product with main_category like 'Appliances'
search_response = es.search(index="products", body={"query": {"match": {"main_category": "Appliances"}}})
search_response.body


/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/2368506357.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_response = es.search(index="products", body={"query": {"match": {"main_category": "Appliances"}}})


{'took': 13,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 164, 'relation': 'eq'},
  'max_score': 4.2607517,
  'hits': [{'_index': 'products',
    '_id': 'B000N40IGY',
    '_score': 4.2607517,
    '_source': {'asin': 'B000N40IGY',
     'title': None,
     'brand': None,
     'image': 'http://ecx.images-amazon.com/images/I/31dC5JloycL._SY300_.jpg',
     'categories': [['Appliances']],
     'main_category': 'Appliances',
     'price': 0.0,
     'model_timestamp': '2023-06-14T12:26:12.129846'}},
   {'_index': 'products',
    '_id': 'B000BPZ7KM',
    '_score': 4.2607517,
    '_source': {'asin': 'B000BPZ7KM',
     'title': None,
     'brand': None,
     'image': 'http://ecx.images-amazon.com/images/I/41I0YHiQHwL._SY300_.jpg',
     'categories': [['Appliances']],
     'main_category': 'Appliances',
     'price': 0.0,
     'model_timestamp': '2023-06-14T12:26:07.394203'}},
   {'_index': 'products',
    '_id': 'B0006FTYG

In [20]:
# get the reviews data
# read the json reviews_appliances file
reviews_file = "/Users/gracechongzuting/Documents/revision_daily/PySpark+PyML/reviews_Appliances.json.gz"

# Replace single quotes with double quotes
with gzip.open(reviews_file, 'rt') as file:
    for line in file:
        pass
        # print(line.replace("'", '"'), end='')

# Read in the reviews file as a DataFrame
reviews_df = spark.read.json(reviews_file)
# create pandas dataframe
reviews_df = reviews_df.toPandas()
reviews_df.head(5)



,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0970408285,"[0, 0]",4.0,Could have been longer though. well made and e...,"12 16, 2013",A27HOWZBUBJ8FF,Steve,Good fit,1387152000
1,7301113188,"[0, 0]",5.0,I like these containers so much i have ordered...,"03 13, 2009",A24HQ894NFSTF5,"Maha Saqfalhait ""shopaholic! ;)""",I Love the Freezer storage line..,1236902400
2,B00002N7HY,"[0, 0]",4.0,"It works, no fires, etc. Why not 5 stars? Ho...","01 7, 2014",AXE83MK90ZEVZ,Strom,expectations achieved.,1389052800
3,B00002NATH,"[0, 0]",5.0,Fast shipping. Works great,"07 20, 2014",A2J7X7ZIH2EWB1,None,Five Stars,1405814400
4,B00002NATH,"[1, 1]",5.0,What can I say? It is the usual Leviton high q...,"06 22, 2010",AJQFNOFTZ7GOX,Barthbill,good product at a good price.,1277164800


In [21]:
# extract the columns we need, asin, reviewerID and overall
reviews_df = reviews_df[['asin', 'reviewerID', 'overall']]
# rename the column 'overall' to 'score'
reviews_df = reviews_df.rename(columns={'overall': 'score'})
# print the dataframe

reviews_df.head(5)



,asin,reviewerID,score
0,0970408285,A27HOWZBUBJ8FF,4.0
1,7301113188,A24HQ894NFSTF5,5.0
2,B00002N7HY,AXE83MK90ZEVZ,4.0
3,B00002NATH,A2J7X7ZIH2EWB1,5.0
4,B00002NATH,AJQFNOFTZ7GOX,5.0


In [22]:
# create spark dataframe of reviews_df

reviews_sdf = spark.createDataFrame(reviews_df)
reviews_sdf.show(5)

+----------+--------------+-----+
|      asin|    reviewerID|score|
+----------+--------------+-----+
|0970408285|A27HOWZBUBJ8FF|  4.0|
|7301113188|A24HQ894NFSTF5|  5.0|
|B00002N7HY| AXE83MK90ZEVZ|  4.0|
|B00002NATH|A2J7X7ZIH2EWB1|  5.0|
|B00002NATH| AJQFNOFTZ7GOX|  5.0|
+----------+--------------+-----+
only showing top 5 rows



In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


# create indexer for asin and review_id column
# purpose of indexer is to convert string to numeric
asin_indexer = StringIndexer(inputCol="asin", outputCol="asin_index")
reviewer_id_indexer = StringIndexer(inputCol="reviewerID", outputCol="reviewer_id_index")

# create pipeline for indexer
# purpose of pipeline is to fit the indexer
indexer_pipeline = Pipeline(stages=[asin_indexer, reviewer_id_indexer])
# fit the pipeline
trans = indexer_pipeline.fit(reviews_sdf).transform(reviews_sdf)
trans.show()







+----------+--------------+-----+----------+-----------------+
|      asin|    reviewerID|score|asin_index|reviewer_id_index|
+----------+--------------+-----+----------+-----------------+
|0970408285|A27HOWZBUBJ8FF|  4.0|    7177.0|          48631.0|
|7301113188|A24HQ894NFSTF5|  5.0|    7178.0|          45922.0|
|B00002N7HY| AXE83MK90ZEVZ|  4.0|    7179.0|         129653.0|
|B00002NATH|A2J7X7ZIH2EWB1|  5.0|    1528.0|          59178.0|
|B00002NATH| AJQFNOFTZ7GOX|  5.0|    1528.0|           8460.0|
|B00002NATH|A3EIP4ZCGM0U67|  5.0|    1528.0|          87135.0|
|B00002NATH|A18N8ND2CH5ZZB|  5.0|    1528.0|          17337.0|
|B00002NATH|A35U08WK3LZ58G|  5.0|    1528.0|          79342.0|
|B00002NATH|A2PMRZGUQQ6V6Q|  5.0|    1528.0|            955.0|
|B00002NATH|A1BFPCIRWQ3WB9|  5.0|    1528.0|          19889.0|
|B00002NATH|A2WQA5YPH8P4SB|  5.0|    1528.0|           5525.0|
|B00002NATH|A2I0LO24RIAAGF|  5.0|    1528.0|          58084.0|
|B00002NATH| AV5YX76EIQOOF|  5.0|    1528.0|         12

23/06/14 12:27:42 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


In [24]:
# als is used for collaborative filtering 
# coldStartStrategy is used to drop rows in dataframe that contains NaN values  
# regParam is used to regularize the model in order to prevent overfitting , 0.01 means 1% of the data is used for regularization
# regularization is used to prevent overfitting
# the higher the regParam, the more regularization is used
# a good regParam is between 0.01 to 0.1
# rank is the number of latent factors in the model, the higher the rank, the more accurate the model is
# a good rank is between 10 to 200  , by default rank is 10
# nonnegative is used to prevent negative values in the model
als = ALS(maxIter=5, rank=VECTOR_DIM, regParam=0.09, userCol="reviewer_id_index", itemCol="asin_index", ratingCol="score",
            coldStartStrategy="drop", nonnegative=True)

# fit the model
model = als.fit(trans)    



23/06/14 12:27:43 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:43 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:44 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:44 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:45 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:46 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:46 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/14 12:27:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/06/14 12:27:47 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:48 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 

In [25]:
# now, we count the itemfactor of the model
# purpose of itemfactor is to get the latent factors of the model
# latent factors are the hidden features that the model has learned from the original data


model.itemFactors.count()

23/06/14 12:27:54 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


11402

In [26]:
# next, we evaluate the regression model

# create evaluator for regression
# prediction column represents the predicted score
# score column represents the actual score


evaluate_als = RegressionEvaluator(metricName="rmse", labelCol="score", predictionCol="prediction")
predict = model.transform(trans)
rmse = evaluate_als.evaluate(predict)
print("Root-mean-square error = " + str(rmse))
predict.show()


23/06/14 12:27:55 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:55 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:56 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:57 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:27:59 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB
23/06/14 12:28:00 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB
23/06/14 12:28:00 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


Root-mean-square error = 0.17475017098658413


23/06/14 12:28:01 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:28:01 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/06/14 12:28:03 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


+----------+--------------+-----+----------+-----------------+----------+
|      asin|    reviewerID|score|asin_index|reviewer_id_index|prediction|
+----------+--------------+-----+----------+-----------------+----------+
|B004Q0OZ6U|A3TAS1AG6FMBQW|  3.0|    4935.0|            108.0| 3.1065826|
|B000MYI3O4|A1W4I5VZN50GRT|  5.0|    1829.0|            688.0| 4.9323006|
|B00482E3F4|A1MGJEVWBTK1N4|  1.0|    1342.0|           2874.0| 0.9766384|
|B002AC705O|A1RGHXPNCLN9YR|  5.0|     833.0|           3173.0|  4.930901|
|B000H2P05Q|A29JLOKP40NEAM|  5.0|     148.0|           4200.0|  4.604648|
|B000H2P05Q|A2QZB3WHWL4NFJ|  5.0|     148.0|           5212.0|   4.63803|
|B004ZI1HZA|A3BXHE4PJ4J2QW|  1.0|    6336.0|           6396.0| 1.0228662|
|B000EMNKOC|A3D3E1LXIEEXYC|  3.0|    7340.0|           6476.0|  2.990402|
|B000H2P05Q| AMFXXAOVV4CPJ|  4.0|     148.0|           8622.0| 3.8388288|
|B0050O1X3O| ASKK1IE5Y22TC|  4.0|    6357.0|           8997.0| 4.0084815|
|B000H2P05Q|A1DY2XW075S22N|  5.0|     

In [27]:
from pyspark.sql.functions import lit, current_timestamp , unix_timestamp
from pyspark.ml.feature import IndexToString
# lit is used to create a new column with a constant value
# current timestamp is used to get the current timestamp
# unix timestamp is used to convert the timestamp to unix timestamp

version = model.uid
unix_time = unix_timestamp(current_timestamp())
product_vector = model.itemFactors.select("id", "features") \
    .withColumnRenamed("id", "asin_index") \
    .withColumnRenamed("features", "model_vector") \
    .withColumn("model_version", lit(version)) \
    .withColumn("model_timestamp", unix_time)

# Show the DataFrame
product_vector.show(3)

# how does model_vector helps in recommendation?
# The model vector captures latent factors that are not explicitly specified but are learned by the recommendation model during training. 
# These latent factors can represent various characteristics of the item, such as its genre, style, popularity
# measuring the distance or similarity between model vectors, 
# the recommendation system can identify items that are similar to each other. Items with similar model vectors are likely to share common characteristics
# The model vector serves as a representation of user preferences, enabling the system to match items with user profiles.
# The model vector also helps address the cold-start problem, where there is limited or no historical data for new items or new users. In such cases, the model vector can be used to estimate the characteristics or features of new items based on their attributes.


+----------+--------------------+----------------+---------------+
|asin_index|        model_vector|   model_version|model_timestamp|
+----------+--------------------+----------------+---------------+
|         0|[0.21924263, 0.28...|ALS_48482ac4832a|     1686716884|
|        10|[0.25532573, 0.10...|ALS_48482ac4832a|     1686716884|
|        20|[0.16128449, 0.15...|ALS_48482ac4832a|     1686716884|
+----------+--------------------+----------------+---------------+
only showing top 3 rows



23/06/14 12:28:04 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


In [28]:


asin_index_meta = [
    f.metadata for f in trans.schema.fields if f.name == "asin_index"]

print(asin_index_meta)
asin_index_labels = asin_index_meta[0]["ml_attr"]["vals"]


reviewerId_converter = IndexToString(inputCol="asin_index", outputCol="asin",   labels=asin_index_labels)
PredictedLabels = reviewerId_converter.transform(product_vector)

# remove the asin_index column and show remaining columns
PredictedLabels = PredictedLabels.drop("asin_index")
PredictedLabels.show(5)




# asin_index_meta is to get the metadata of asin_index column
# metadata because we need to get the labels of the asin_index column
# ml_attr represents the metadata of the column
# vals represents the labels of the column, labels are the values of the column




[{'ml_attr': {'vals': ['B000AST3AK', 'B000UW2DTE', 'B001XW8KW4', 'B000DZGN7Q', 'B0006MQCA4', 'B000KI3O1O', 'B0002UMYDE', 'B0014X7B54', 'B000SX4C7Q', 'B0009793KC', 'B003N1ZSYG', 'B001TJ3J9K', 'B00004U9JO', 'B001TJ5380', 'B000G837TW', 'B000O3I2CG', 'B00E37TQLA', 'B005MMSTNG', 'B001BOTNCU', 'B0042ACZU2', 'B000NCTOUM', 'B0045LLC7K', 'B002UYSHMM', 'B000UW3RK8', 'B0049IA8QK', 'B004W5WAMU', 'B004RQDY60', 'B001DHKBXU', 'B007NXPAQG', 'B004I08CEO', 'B0029CZQIS', 'B000BTDNEK', 'B001DHLGOS', 'B000HE5DNI', 'B00DM8J11Q', 'B00E37TQRO', 'B002C8HR9A', 'B0002YTLZY', 'B002A0KB3Y', 'B001DPXZBM', 'B004JV8ZUS', 'B001BORKYS', 'B004MX8XO6', 'B001DPOMMI', 'B000RDAORO', 'B00CBG2K7C', 'B0089VAAF2', 'B00126NABC', 'B00074TB9U', 'B004DNWVOI', 'B0042U16YI', 'B003BIGDLI', 'B002MPLYEW', 'B0032AMC4C', 'B0002GTTRC', 'B0007W1EA6', 'B002JAKRAM', 'B000FFIYEE', 'B0017Y3GGI', 'B00AW04ZMM', 'B000KFZEBK', 'B00067YLKW', 'B0007ZYUAE', 'B005AR76M8', 'B005GM942C', 'B001ICYB2M', 'B0053F80JA', 'B004H3XQPM', 'B00006IV17', 'B0007VWEQA

23/06/14 12:28:05 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB


In [29]:
predictedlabels = PredictedLabels.count()
print(f"predicted labels count: {predictedlabels}")

23/06/14 12:28:05 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


predicted labels count: 11402


In [30]:


# Convert DataFrame to Pandas DataFrame
pandas_df = PredictedLabels.toPandas()

# change model_vector column dimension from 10 to 25 adding 0s
# pandas_df['model_vector'] = pandas_df['model_vector'].apply(lambda x: np.append(x, np.zeros(15)))

# Bulk index the Pandas DataFrame to Elasticsearch
documents = pandas_df.to_dict(orient='records')

for doc in documents:
    id = doc['asin']
    _ = es.get(index="products", id=id)
    res = es.update(index="products", id=id, body={'doc': doc})




23/06/14 12:28:06 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB
/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/2702437792.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.update(index="products", id=id, body={'doc': doc})


In [31]:
PredictedLabels.show(5)

+--------------------+----------------+---------------+----------+
|        model_vector|   model_version|model_timestamp|      asin|
+--------------------+----------------+---------------+----------+
|[0.21924263, 0.28...|ALS_48482ac4832a|     1686716991|B000AST3AK|
|[0.25532573, 0.10...|ALS_48482ac4832a|     1686716991|B003N1ZSYG|
|[0.16128449, 0.15...|ALS_48482ac4832a|     1686716991|B000NCTOUM|
|[0.17999192, 0.31...|ALS_48482ac4832a|     1686716991|B0029CZQIS|
|[0.30785376, 0.08...|ALS_48482ac4832a|     1686716991|B004JV8ZUS|
+--------------------+----------------+---------------+----------+
only showing top 5 rows



23/06/14 12:29:51 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB


In [32]:
from IPython.display import Image, HTML, display
def vector_query(query_vec, category,vector_field, cosine=False):
    """
    Construct an Elasticsearch script score query using `dense_vector` fields
    
    The script score query takes as parameters the query vector (as a Python list)
    
    Parameters
    ----------
    query_vec : list
        The query vector
    vector_field : str
        The field name in the document against which to score `query_vec`
    q : str, optional
        Query string for the search query (default: '*' to search across all documents)
    cosine : bool, optional
        Whether to compute cosine similarity. If `False` then the dot product is computed (default: False)
     
    Note: Elasticsearch cannot rank negative scores. Therefore, in the case of the dot product, a sigmoid transform
    is applied. In the case of cosine similarity, 1.0 is added to the score. In both cases, documents with no 
    factor vectors are ignored by applying a 0.0 score.
    
    The query vector passed in will be the user factor vector (if generating recommended items for a user)
    or product factor vector (if generating similar items for a given item)
    """
    
    if cosine:
        score_fn = "doc['{v}'].size() == 0 ? 0 : cosineSimilarity(params.vector, '{v}') + 1.0"
    else:
        score_fn = "doc['{v}'].size() == 0 ? 0 : sigmoid(1, Math.E, -dotProduct(params.vector, '{v}'))"
       
    score_fn = score_fn.format(v=vector_field, fn=score_fn)
    
    return {
    "query": {
        "script_score": {
            "query" : { 
                "bool" : {
                      "filter" : {
                            "term" : {
                              "main_category" : category
                            }
                        }
                }
            },
            "script": {
                "source": score_fn,
                "params": {
                    "vector": query_vec
                }
            }
        }
    }
}


def get_similar(the_id, num=10, index="products", vector_field='model_vector'):
    """
    Given a item id, execute the recommendation script score query to find similar items,
    ranked by cosine similarity. We return the `num` most similar, excluding the item itself.
    """
    response = es.get(index=index, id=the_id)
    src = response['_source']
    if vector_field in src:
        query_vec = src[vector_field]
        category = src['main_category']
        q = vector_query(query_vec, category,vector_field, cosine=True)
#         print(q)
        results = es.search(index=index, body=q)
        hits = results['hits']['hits']
        return src,hits[1:num+1]

def display_similar(the_id, num=10, es_index="products"):
    """
    Display query product, together with similar product and similarity scores, in a table
    """
    product, recs = get_similar(the_id, num, es_index)
       
    display(HTML("<h2>Get similar products for:</h2>"))
    display(HTML("<h4>%s (ASIN - %s)</h4>" % (product['title'], product['asin'])))
    display(HTML("<br>"))
    display(HTML("<h2>People who liked this product also liked these:</h2>"))
    sim_html = "<table border=0>"
    i = 0
    pd_data = []
    for rec in recs:
        r_id = rec['_id']
        r_score = rec['_score']
        r_title = rec['_source']['title']
        r = {}
        r['asin'] = rec['_source']['asin']
        r['title'] = r_title
        r['score'] = r_score
        pd_data.append(r)
        r_im_url = rec['_source']['image'] or ''
        sim_html += "<tr><td><h5>%s - %s</h5><img src=%s width=150></img></td><td><h5>%2.3f</h5></td></tr>" % (r_id, r_title, r_im_url, r_score)
        i += 1
    sim_html += "</table>"
    pd.set_option('display.max_colwidth', -1) 
    pd_df = pd.DataFrame (pd_data)
    display(HTML(pd_df.to_html()))
    display(HTML(sim_html))

In [33]:
data = display_similar('B0075CU8MG', num=5)

/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/7932373.py:69: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index=index, body=q)


/var/folders/3v/prng965x6vg3z8l8gf2j4pmw0000gn/T/ipykernel_1702/7932373.py:99: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,asin,title,score
0,B000UVUPPY,None,1.742133
1,B005HF5H5Q,None,1.716442
2,B000UVUQJ4,None,1.710652
3,B002WS5FUS,Maytag MEC4536WB 36&quot; Black Electric Coil Cooktop,1.710645
4,B002C7PYFA,None,1.704245


B000UVUPPY - None,1.742
B005HF5H5Q - None,1.716
B000UVUQJ4 - None,1.711
"B002WS5FUS - Maytag MEC4536WB 36"" Black Electric Coil Cooktop",1.711
B002C7PYFA - None,1.704


In [34]:
es.indices.refresh(index="products")

# es.search(index="products", body={"query": {"match": {"main_category": "Cooktops"}}})

# es.get(index="products", id="962")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})